In [2]:
import pandas as pd
import re
import re
from collections import defaultdict
import html;
import csv

p = {
    "label": re.compile(r'^\s*([A-Za-z_]\w*)\s*:\s*(?!:)'),
    "case": re.compile(r'^\s*case\b[^:]*:\s*$'),
    "default": re.compile(r'^\s*default\s*:\s*$'),
    "iff": re.compile(r'^\s*if\s*\('),
    "else": re.compile(r'^\s*else\b'),
    "for": re.compile(r'^\s*for\s*\('),
    "while": re.compile(r'^\s*while\s*\('),
    "do": re.compile(r'^\s*do\s*(?:\{)?\s*$'),
    "goto": re.compile(r'\bgoto\s+([A-Za-z_]\w*)\s*;'),
    "break": re.compile(r'^\s*break\s*;'),
    "cont": re.compile(r'^\s*continue\s*;'),
    "ret": re.compile(r'^\s*return\b'),
    "func": re.compile(r'^\s*(?:[A-Za-z_]\w*\s+)*[A-Za-z_]\w*\s*\([^;]*\)\s*\{?\s*$'),
}

## Functions to read the code and analyze the keyword so find the leaders

In [3]:
def readfile(fn):
    with open(fn, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

def structural_only(s):
    t = s.strip()
    return t in ("{","}","};")

def next_stmt(lines, j):
    k = j + 1
    while k < len(lines):
        t = lines[k]
        if t.strip() and not t.lstrip().startswith("#") and not structural_only(t):
            return k
        k += 1
    return None

def first_stmt_in_block(lines, start_line):
    k = start_line
    if "{" not in lines[k]:
        while k < len(lines) and "{" not in lines[k]:
            k += 1
    k += 1
    while k < len(lines):
        t = lines[k]
        if t.strip() and not t.lstrip().startswith("#") and not structural_only(t):
            return k
        k += 1
    return None

def leaders_from_src(src):
    lines = src.splitlines()
    ls = set()
    labels = {}

    for i, s in enumerate(lines):
        if p["func"].match(s):
            ls.add(i)
            k = first_stmt_in_block(lines, i)
            if k is not None:
                ls.add(k)

    for i, s in enumerate(lines):
        if s.strip() and not s.lstrip().startswith("#"):
            ls.add(i)
            break

    for i, s in enumerate(lines):
        m = p["label"].match(s)
        if m:
            labels[m.group(1)] = i
            ls.add(i)
        if p["case"].match(s) or p["default"].match(s):
            ls.add(i)

    header_keys = ("iff","else","for","while","do")
    for i, s in enumerate(lines):
        if any(p[k].search(s) for k in header_keys):
            ls.add(i)
            k = next_stmt(lines, i)
            if k is not None:
                ls.add(k)

        if p["break"].search(s) or p["cont"].search(s) or p["ret"].search(s):
            k = next_stmt(lines, i)
            if k is not None:
                ls.add(k)

        if s.strip() == "}":
            k = next_stmt(lines, i)
            if k is not None:
                ls.add(k)

        m = p["goto"].search(s)
        if m and m.group(1) in labels:
            ls.add(labels[m.group(1)])

    idxs = sorted(ls)
    return idxs, lines


## Generating blocks from leaders

In [4]:
def nonstruct(s):
    t = s.strip()
    return t and t not in ("{","}","};") and not t.startswith("#")

def build_blocks_from_leaders(lines, leaders):
    leaders = sorted(set(leaders))
    leaders.append(len(lines))
    blocks = []
    for idx in range(len(leaders)-1):
        start = leaders[idx]
        end = leaders[idx+1]
        s = start
        while s < end and not nonstruct(lines[s]):
            s += 1
        e = end
        while e > s and not nonstruct(lines[e-1]):
            e -= 1
        if s < e:
            blocks.append((s, e))
    return blocks

## Generating Edges

In [ ]:
is_branch_hdr = re.compile(r'^\s*(if|while|for)\b')
is_return     = re.compile(r'^\s*return\b')
is_break      = re.compile(r'^\s*break\s*;')
is_continue   = re.compile(r'^\s*continue\s*;')

def nonstruct(s):
    t = s.strip()
    return t and t not in ("{","}","};") and not t.startswith("#")

def last_stmt_line(lines, s, e):
    for k in range(e - 1, s - 1, -1):
        if nonstruct(lines[k]):
            return k
    return None
    return N, E, CC

def build_line_to_block(blocks):
    line2b = {}
    for bi, (s,e) in enumerate(blocks):
        for k in range(s, e):
            line2b[k] = bi
    return line2b

def find_follow_block(lines, blocks, hdr_line, line2b):
    depth = 0
    i = hdr_line
    opened = False
    while i < len(lines):
        if '{' in lines[i]: 
            depth += 1; opened = True
        if '}' in lines[i]:
            depth -= 1
            if opened and depth == 0: 
                k = next_stmt(lines, i)
                if k is not None and k in line2b:
                    return line2b[k]
                break
        i += 1
    # fallback: next block
    b = line2b.get(hdr_line, None)
    return (b+1) if b is not None and b+1 < len(blocks) else None

def find_body_block(lines, hdr_line, line2b):
    k = first_stmt_in_block(lines, hdr_line)
    return line2b.get(k, None) if k is not None else None

def build_cfg_edges(lines, blocks):
    names = [f"B{i}" for i in range(len(blocks))]
    edges = []
    line2b = build_line_to_block(blocks)
    loop_headers = []  

    for i, (s,e) in enumerate(blocks):
        name = names[i]
        k = last_stmt_line(lines, s, e)
        if k is None:
            if i+1 < len(blocks): edges.append((name, names[i+1], "fall"))
            continue

        tail = lines[k]

        if is_return.match(tail):
            continue

        if is_branch_hdr.match(tail):
            body_b = find_body_block(lines, k, line2b)
            follow_b = find_follow_block(lines, blocks, k, line2b)
            if body_b is not None:  edges.append((name, names[body_b], "true"))
            if follow_b is not None: edges.append((name, names[follow_b], "false"))
            continue

        if is_continue.match(tail):
            if loop_headers:
                hdr_b = loop_headers[-1][1]
                edges.append((name, names[hdr_b], "continue"))
            continue

        if is_break.match(tail):
            if loop_headers:
                hdr_line = last_stmt_line(lines, *blocks[loop_headers[-1][1]])
                out_b = find_follow_block(lines, blocks, hdr_line, line2b)
                if out_b is not None:
                    edges.append((name, names[out_b], "break"))
            continue

        if i+1 < len(blocks):
            edges.append((name, names[i+1], "fall"))

    return names, edges


In [6]:
assign_pat = re.compile(r'\b([A-Za-z_]\w*(?:\[[^\]]+\])?)\s*=')

def nonstruct(s):
    t = s.strip()
    return t and t not in ("{","}","};") and not t.startswith("#")

def block_names(blocks):
    return [f"B{i}" for i in range(len(blocks))]

def collect_definitions(lines, blocks):
    defs = []
    names = block_names(blocks)
    for bi,(s,e) in enumerate(blocks):
        bname = names[bi]
        for li in range(s, e):
            sline = lines[li]
            if not nonstruct(sline):
                continue
            parts = [p for p in sline.split(';') if p.strip()]
            for stmt in parts:
                if '=' in stmt and '==' not in stmt:
                    m = assign_pat.search(stmt)
                    if m:
                        var = m.group(1).strip()
                        defs.append((None, var, li, stmt.strip(), bname))
    out = []
    for k,(nid,var,li,txt,bname) in enumerate(defs, start=1):
        out.append((f"D{k}", var, li, txt, bname))
    return out

def gen_kill_per_block(defs, blocks):
    names = block_names(blocks)
    all_by_var = defaultdict(set)
    for Dk,var,li,txt,b in defs:
        all_by_var[var].add(Dk)
    gen = {b:set() for b in names}
    for Dk,var,li,txt,b in defs:
        gen[b].add(Dk)
    kill = {b:set() for b in names}
    for b in names:
        vars_in_b = set(var for (Dk,var,li,txt,bb) in defs if bb == b)
        kset = set()
        for v in vars_in_b:
            kset |= (all_by_var[v] - gen[b])
        kill[b] = kset
    return gen, kill

def preds(edges):
    P = defaultdict(set)
    for u,v,lbl in edges:
        P[v].add(u)
    return P

def reaching_definitions(blocks, edges, gen, kill):
    names = block_names(blocks)
    IN = {b:set() for b in names}
    OUT = {b:set() for b in names}
    P = preds(edges)
    changed = True
    it = 0
    logs = []
    while changed and it < 100:
        changed = False
        it += 1
        rows = []
        for b in names:
            in_new = set()
            for p in P[b]:
                in_new |= OUT[p]
            out_new = gen[b] | (in_new - kill[b])
            rows.append((b, sorted(gen[b]), sorted(kill[b]), sorted(in_new), sorted(out_new)))
            if in_new != IN[b] or out_new != OUT[b]:
                changed = True
            IN[b] = in_new
            OUT[b] = out_new
        logs.append(rows)
    return IN, OUT, logs

## Graphviz dot file generation

In [7]:
def stmt_lines(lines, s, e):
    out = []
    for k in range(s, e):
        parts = [p.strip() for p in lines[k].rstrip().split(';') if p.strip()]
        for p in parts:
            out.append(p + ";")
    return out

def html_block_label(lines, s, e, name):
    esc = lambda x: html.escape(x, quote=False)
    rows = "".join(f"<tr><td align='left'>{esc(p)}</td></tr>" for p in stmt_lines(lines, s, e))
    header = f"<tr><td bgcolor='#eeeeee'><b>{esc(name)} [{s},{e})</b></td></tr>"
    if not rows:
        rows = "<tr><td align='left'>(empty);</td></tr>"
    return f"<<table border='0' cellborder='1' cellspacing='0'>{header}{rows}</table>>"

def export_cfg_dot_html(lines, blocks, edges, program_name="code"):
    names = [f"B{i}" for i in range(len(blocks))]
    with open("cfg.dot", "w", encoding="utf-8") as f:
        f.write("digraph CFG {\n")
        f.write('  graph [fontname="Courier", nodesep=0.35, ranksep=0.4];\n')
        f.write('  node [shape=plaintext, fontname="Courier"];\n')
        f.write('  edge [fontname="Courier"];\n')
        for i,(s,e) in enumerate(blocks):
            label = html_block_label(lines, s, e, names[i])
            f.write(f"  {names[i]} [label={label}];\n")
        for u,v,lbl in edges:
            f.write(f'  {u} -> {v} [label="{lbl}"];\n')
        f.write(f'  labelloc="t";\n  label="CFG for {program_name}";\n')
        f.write("}\n")


### Generating CSV files for reaching definitions

In [8]:
def export_rd_csv(defs, gen, kill, IN, OUT, prefix="rd"):
    with open(f"{prefix}_defs.csv", "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["Dk","var","line","block","stmt"])
        for Dk,var,li,txt,b in defs:
            w.writerow([Dk,var,li,b,txt])
    order = sorted(gen.keys(), key=lambda x:int(x[1:]))
    with open(f"{prefix}_sets.csv", "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["Block","gen","kill","IN","OUT"])
        for b in order:
            w.writerow([
                b,
                " ".join(sorted(gen[b])),
                " ".join(sorted(kill[b])),
                " ".join(sorted(IN[b])),
                " ".join(sorted(OUT[b]))
            ])

### Printing the results

In [9]:
def print_Dk_map(defs):
    print("Definition IDs:")
    for Dk,var,li,txt,b in defs:
        print(f"{Dk}: var={var}, line={li}, block={b}, stmt=[{txt}]")

def print_gen_kill(gen, kill):
    order = sorted(gen.keys(), key=lambda x:int(x[1:]))
    print("\nGEN/KILL:")
    for b in order:
        print(f"{b}: gen={sorted(gen[b])} kill={sorted(kill[b])}")

def print_final_in_out(IN, OUT):
    order = sorted(IN.keys(), key=lambda x:int(x[1:]))
    print("\nFinal IN/OUT:")
    for b in order:
        print(f"{b}: IN={sorted(IN[b])} OUT={sorted(OUT[b])}")


# Code1.c


In [18]:
src = readfile("code1.c")
idxs, lines = leaders_from_src(src)
blocks = build_blocks_from_leaders(lines, idxs)

print()
print("-" * 80)
print("Identified leader lines")
print("-" * 80)
print()

for a in idxs:
    print(a + 1, end = " ")
    print(lines[a])
    # print()

print()
print("-" * 80)
print("Basic Blocks")
print("-" * 80)
print()

print("\nbasic blocks:", len(blocks))
for bi, (s, e) in enumerate(blocks):
    print(f"B{bi} [{s},{e})")
    for k in range(s, e):
        print(f"  {k}: {lines[k]}")

# generting edges

print()
print("-" * 80)
print("Constructed Edges")
print("-" * 80)
print()

names, edges = build_cfg_edges(lines, blocks)
print("\nedges:")
for u,v,lbl in edges:
    print(f"{u} -> {v} ({lbl})")

print()
print("-" * 80)
print("Obtained Metrics")
print("-" * 80)
print()

N,E,CC = cfg_metrics(len(blocks), edges)
print(f"\nmetrics: N={N}, E={E}, CC={CC}")

# reaching DFA

print()
print("-" * 80)
print("Definition for Reaching")
print("-" * 80)
print()

defs = collect_definitions(lines, blocks)
print(f"\nDefinitions ({len(defs)}):")
for Dk,var,li,txt,b in defs[:20]:
    print(f"{Dk} @ line {li} in {b}: {var} = ... ; [{txt}]")
if len(defs) > 20:
    print("...")

print()
print("-" * 80)
print("Obtaining Generation and Killings")
print("-" * 80)
print()

gen, kill = gen_kill_per_block(defs, blocks)
print("\nGEN/KILL per block:")
for b in sorted(gen.keys(), key=lambda x:int(x[1:])):
    print(b, "gen:", sorted(gen[b]), "kill:", sorted(kill[b]))

print()
print("-" * 80)
print("Iterations (Capped at 100)")
print("-" * 80)
print()



IN, OUT, logs = reaching_definitions(blocks, edges, gen, kill)
print("\nReaching Definitions Iterations:")
for it, rows in enumerate(logs, start=1):
    print(f"\nIteration {it}")
    for b,g,k,ins,outs in rows:
        print(f"{b} | gen:{g} kill:{k} in:{ins} out:{outs}")

print()
print("-" * 80)
print("Final IN and OUT")
print("-" * 80)
print()


print("\nFinal IN/OUT:")
for b in sorted(IN.keys(), key=lambda x:int(x[1:])):
    print(f"{b} IN:{sorted(IN[b])} OUT:{sorted(OUT[b])}")

print()
print("-" * 80)
print("Exporting as DOT File and Image Generation")
print("-" * 80)
print()


# exporting the dot files
export_cfg_dot_html(lines, blocks, edges, program_name="code1.c")

!dot -Tpng cfg.dot -o cfg.png

print()
print("-" * 80)
print("Exporting as CSV File")
print("-" * 80)
print()


export_rd_csv(defs, gen, kill, IN, OUT, prefix="code1")


--------------------------------------------------------------------------------
Identified leader lines
--------------------------------------------------------------------------------

3 int main() {
4     int n = 200;
30     for (i = 0; i < n; i++) {
31         data[i] = (i * alpha + i * i * beta + gamma) % mod;
36         if (data[i] > 50) {
37             data[i] = data[i] - 30;
39         else if (data[i] < -20) {
40             data[i] = data[i] + 10;
42         else {
43             data[i] = data[i] + 5;
46         if (data[i] % 2 == 0) {
47             pos++;
49         else if (data[i] % 3 == 0) {
50             neg++;
52         else {
53             zero++;
56         total += data[i];
59     for (i = 0; i < n; i++) {
60         int t = data[i];
63         if (t > 70) {
64             t = t - (i % 7);
66         else if (t < 20) {
67             t = t + (i % 11);
69         else {
70             t = t * 2 - (i % 9);
73         temp[i] = t;
77     for (iteration = 0; itera

#### Exploratory research analysis

In [11]:
# usage after Step 3:

print_Dk_map(defs)
print_gen_kill(gen, kill)
print_final_in_out(IN, OUT)

Definition IDs:
D1: var=n, line=3, block=B1, stmt=[int n = 200]
D2: var=i, line=8, block=B1, stmt=[int i = 0, j = 0, k = 0]
D3: var=total, line=9, block=B1, stmt=[int total = 0]
D4: var=pos, line=10, block=B1, stmt=[int pos = 0, neg = 0, zero = 0]
D5: var=rangeLow, line=11, block=B1, stmt=[int rangeLow = -150]
D6: var=rangeHigh, line=12, block=B1, stmt=[int rangeHigh = 150]
D7: var=alpha, line=13, block=B1, stmt=[int alpha = 3, beta = 7, gamma = 5]
D8: var=mod, line=14, block=B1, stmt=[int mod = 97]
D9: var=balance, line=15, block=B1, stmt=[int balance = 0]
D10: var=flag, line=16, block=B1, stmt=[int flag = 0]
D11: var=adjust, line=17, block=B1, stmt=[int adjust = 0]
D12: var=iteration, line=18, block=B1, stmt=[int iteration = 0]
D13: var=offset, line=19, block=B1, stmt=[int offset = 0]
D14: var=shift, line=20, block=B1, stmt=[int shift = 1]
D15: var=rolling, line=21, block=B1, stmt=[int rolling = 0]
D16: var=bigSum, line=22, block=B1, stmt=[int bigSum = 0]
D17: var=result, line=23, bl

# code2.c


In [19]:
src = readfile("code2.c")
idxs, lines = leaders_from_src(src)
blocks = build_blocks_from_leaders(lines, idxs)

print()
print("-" * 80)
print("Identified leader lines")
print("-" * 80)
print()

for a in idxs:
    print(a + 1, end = " ")
    print(lines[a])
    # print()

print()
print("-" * 80)
print("Basic Blocks")
print("-" * 80)
print()

print("\nbasic blocks:", len(blocks))
for bi, (s, e) in enumerate(blocks):
    print(f"B{bi} [{s},{e})")
    for k in range(s, e):
        print(f"  {k}: {lines[k]}")

# generting edges

print()
print("-" * 80)
print("Constructed Edges")
print("-" * 80)
print()

names, edges = build_cfg_edges(lines, blocks)
print("\nedges:")
for u,v,lbl in edges:
    print(f"{u} -> {v} ({lbl})")

print()
print("-" * 80)
print("Obtained Metrics")
print("-" * 80)
print()

N,E,CC = cfg_metrics(len(blocks), edges)
print(f"\nmetrics: N={N}, E={E}, CC={CC}")

# reaching DFA

print()
print("-" * 80)
print("Definition for Reaching")
print("-" * 80)
print()

defs = collect_definitions(lines, blocks)
print(f"\nDefinitions ({len(defs)}):")
for Dk,var,li,txt,b in defs[:20]:
    print(f"{Dk} @ line {li} in {b}: {var} = ... ; [{txt}]")
if len(defs) > 20:
    print("...")

print()
print("-" * 80)
print("Obtaining Generation and Killings")
print("-" * 80)
print()

gen, kill = gen_kill_per_block(defs, blocks)
print("\nGEN/KILL per block:")
for b in sorted(gen.keys(), key=lambda x:int(x[1:])):
    print(b, "gen:", sorted(gen[b]), "kill:", sorted(kill[b]))

print()
print("-" * 80)
print("Iterations (Capped at 100)")
print("-" * 80)
print()



IN, OUT, logs = reaching_definitions(blocks, edges, gen, kill)
print("\nReaching Definitions Iterations:")
for it, rows in enumerate(logs, start=1):
    print(f"\nIteration {it}")
    for b,g,k,ins,outs in rows:
        print(f"{b} | gen:{g} kill:{k} in:{ins} out:{outs}")

print()
print("-" * 80)
print("Final IN and OUT")
print("-" * 80)
print()


print("\nFinal IN/OUT:")
for b in sorted(IN.keys(), key=lambda x:int(x[1:])):
    print(f"{b} IN:{sorted(IN[b])} OUT:{sorted(OUT[b])}")

print()
print("-" * 80)
print("Exporting as DOT File and Image Generation")
print("-" * 80)
print()


# exporting the dot files
export_cfg_dot_html(lines, blocks, edges, program_name="code1.c")

!dot -Tpng cfg.dot -o cfg2.png

print()
print("-" * 80)
print("Exporting as CSV File")
print("-" * 80)
print()


export_rd_csv(defs, gen, kill, IN, OUT, prefix="code1")


--------------------------------------------------------------------------------
Identified leader lines
--------------------------------------------------------------------------------

3 int main() {
4     int i = 0;
10     for (i = 0; i < n; i++) {
11         arr[i] = (i * i + 3 * i + 7) % 97;
13     int sum = 0;
14     for (i = 0; i < n; i++) {
15         sum = sum + arr[i];
16         if (arr[i] % 2 == 0) {
17             arr[i] = arr[i] / 2;
19         else {
20             arr[i] = arr[i] * 3 + 1;
23     int even = 0;
25     for (i = 0; i < n; i++) {
26         if (arr[i] % 2 == 0) {
27             even = even + 1;
29         else {
30             odd = odd + 1;
33     int threshold = (sum / n) % 50;
35     for (i = 0; i < n; i++) {
36         if (arr[i] > threshold) {
37             greater = greater + 1;
39         else {
40             arr[i] = arr[i] + threshold / 2;
43     int iteration = 0;
44     while (iteration < 100) {
45         for (i = 0; i < n; i++) {
46          

In [13]:
# usage after Step 3:

print_Dk_map(defs)
print_gen_kill(gen, kill)
print_final_in_out(IN, OUT)

Definition IDs:
D1: var=i, line=3, block=B1, stmt=[int i = 0]
D2: var=j, line=4, block=B1, stmt=[int j = 0]
D3: var=k, line=5, block=B1, stmt=[int k = 0]
D4: var=n, line=6, block=B1, stmt=[int n = 60]
D5: var=i, line=9, block=B2, stmt=[for (i = 0]
D6: var=arr[i], line=10, block=B3, stmt=[arr[i] = (i * i + 3 * i + 7) % 97]
D7: var=sum, line=12, block=B4, stmt=[int sum = 0]
D8: var=i, line=13, block=B5, stmt=[for (i = 0]
D9: var=sum, line=14, block=B6, stmt=[sum = sum + arr[i]]
D10: var=arr[i], line=16, block=B8, stmt=[arr[i] = arr[i] / 2]
D11: var=arr[i], line=19, block=B10, stmt=[arr[i] = arr[i] * 3 + 1]
D12: var=even, line=22, block=B11, stmt=[int even = 0]
D13: var=odd, line=23, block=B11, stmt=[int odd = 0]
D14: var=i, line=24, block=B12, stmt=[for (i = 0]
D15: var=even, line=26, block=B14, stmt=[even = even + 1]
D16: var=odd, line=29, block=B16, stmt=[odd = odd + 1]
D17: var=threshold, line=32, block=B17, stmt=[int threshold = (sum / n) % 50]
D18: var=greater, line=33, block=B17, s

# Code3.c

In [20]:
src = readfile("code3.c")
idxs, lines = leaders_from_src(src)
blocks = build_blocks_from_leaders(lines, idxs)

print()
print("-" * 80)
print("Identified leader lines")
print("-" * 80)
print()

for a in idxs:
    print(a + 1, end = " ")
    print(lines[a])
    # print()

print()
print("-" * 80)
print("Basic Blocks")
print("-" * 80)
print()

print("\nbasic blocks:", len(blocks))
for bi, (s, e) in enumerate(blocks):
    print(f"B{bi} [{s},{e})")
    for k in range(s, e):
        print(f"  {k}: {lines[k]}")

# generting edges

print()
print("-" * 80)
print("Constructed Edges")
print("-" * 80)
print()

names, edges = build_cfg_edges(lines, blocks)
print("\nedges:")
for u,v,lbl in edges:
    print(f"{u} -> {v} ({lbl})")

print()
print("-" * 80)
print("Obtained Metrics")
print("-" * 80)
print()

N,E,CC = cfg_metrics(len(blocks), edges)
print(f"\nmetrics: N={N}, E={E}, CC={CC}")

# reaching DFA

print()
print("-" * 80)
print("Definition for Reaching")
print("-" * 80)
print()

defs = collect_definitions(lines, blocks)
print(f"\nDefinitions ({len(defs)}):")
for Dk,var,li,txt,b in defs[:20]:
    print(f"{Dk} @ line {li} in {b}: {var} = ... ; [{txt}]")
if len(defs) > 20:
    print("...")

print()
print("-" * 80)
print("Obtaining Generation and Killings")
print("-" * 80)
print()

gen, kill = gen_kill_per_block(defs, blocks)
print("\nGEN/KILL per block:")
for b in sorted(gen.keys(), key=lambda x:int(x[1:])):
    print(b, "gen:", sorted(gen[b]), "kill:", sorted(kill[b]))

print()
print("-" * 80)
print("Iterations (Capped at 100)")
print("-" * 80)
print()



IN, OUT, logs = reaching_definitions(blocks, edges, gen, kill)
print("\nReaching Definitions Iterations:")
for it, rows in enumerate(logs, start=1):
    print(f"\nIteration {it}")
    for b,g,k,ins,outs in rows:
        print(f"{b} | gen:{g} kill:{k} in:{ins} out:{outs}")

print()
print("-" * 80)
print("Final IN and OUT")
print("-" * 80)
print()


print("\nFinal IN/OUT:")
for b in sorted(IN.keys(), key=lambda x:int(x[1:])):
    print(f"{b} IN:{sorted(IN[b])} OUT:{sorted(OUT[b])}")

print()
print("-" * 80)
print("Exporting as DOT File and Image Generation")
print("-" * 80)
print()


# exporting the dot files
export_cfg_dot_html(lines, blocks, edges, program_name="code1.c")

!dot -Tpng cfg.dot -o cfg3.png

print()
print("-" * 80)
print("Exporting as CSV File")
print("-" * 80)
print()


export_rd_csv(defs, gen, kill, IN, OUT, prefix="code1")


--------------------------------------------------------------------------------
Identified leader lines
--------------------------------------------------------------------------------

3 int main() {
4     int i = 0;
16     for (i = 0; i < n; i++) {
17         arr[i] = (i * 13 + 5 * (i % 7) + 11) % 97;
19     for (i = 0; i < n; i++) {
20         total = total + arr[i];
23     int average = total / n;
25     for (i = 0; i < n; i++) {
26         int value = arr[i];
27         if (value > pivot) {
28             value = value - (value / 5);
35             for (j = 0; j < 3; j++) {
36                 value = (value + j + i) % 91;
38                 if (balance % 7 == 0) {
39                     value = value + 2;
41                 else {
42                     value = value - 3;
45             arr[i] = value;
47         else {
48             value = value + (pivot - value) / 2;
53             for (j = 0; j < 2 * (i % 3 + 1); j++) {
54                 value = (value + 5 + j) % 97;
55   

In [15]:
# usage after Step 3:

print_Dk_map(defs)
print_gen_kill(gen, kill)
print_final_in_out(IN, OUT)

Definition IDs:
D1: var=i, line=3, block=B1, stmt=[int i = 0]
D2: var=j, line=4, block=B1, stmt=[int j = 0]
D3: var=n, line=5, block=B1, stmt=[int n = 60]
D4: var=total, line=8, block=B1, stmt=[int total = 0]
D5: var=x, line=9, block=B1, stmt=[int x = 3]
D6: var=y, line=10, block=B1, stmt=[int y = 7]
D7: var=z, line=11, block=B1, stmt=[int z = 9]
D8: var=balance, line=12, block=B1, stmt=[int balance = 0]
D9: var=iteration, line=13, block=B1, stmt=[int iteration = 0]
D10: var=mode, line=14, block=B1, stmt=[int mode = 1]
D11: var=i, line=15, block=B2, stmt=[for (i = 0]
D12: var=arr[i], line=16, block=B3, stmt=[arr[i] = (i * 13 + 5 * (i % 7) + 11) % 97]
D13: var=i, line=18, block=B4, stmt=[for (i = 0]
D14: var=total, line=19, block=B5, stmt=[total = total + arr[i]]
D15: var=aux[i], line=20, block=B5, stmt=[aux[i] = arr[i] % 10]
D16: var=average, line=22, block=B6, stmt=[int average = total / n]
D17: var=pivot, line=23, block=B6, stmt=[int pivot = (average % 20) + 10]
D18: var=i, line=24, 